## Chatbot Agent
Agent base chatbot using Amazon Bedrock and LangGraph.
This bot has memeory capablilities to remember the previous prompts in the same conversation.

In [ ]:
import boto3
import json
import logging

from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser

boto3.set_stream_logger('', logging.ERROR)

# Initialize AWS Bedrock client
bedrock_client = boto3.client(
    service_name='bedrock-runtime'
    ,region_name='ap-southeast-2'
)

llm = ChatBedrock(
    client=bedrock_client,
    model_id="amazon.nova-pro-v1:0",  
    model_kwargs={
        "temperature": 0.3,  # Same temperature as original
        "maxTokenCount": 1000
    }
)

load LangSmith environment

In [ ]:
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode, tools_condition

from typing_extensions import TypedDict
from typing import Annotated

In [ ]:
%load_ext dotenv
%dotenv ../../.env

In [ ]:
from langchain_community.tools import TavilySearchResults
travily_search = TavilySearchResults(max_results=2)

In [ ]:
# current date and time
import datetime
from langchain_core.tools import tool

@tool
def get_current_date():
    """
    Get the current date and time.
    Use this tool first for any tile-based queries.
    """
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

Bind the LLM to tools:

In [ ]:
llm_with_tools = llm.bind_tools([get_current_date, travily_search])

In [ ]:
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

Define the nodes:

In [ ]:
def chatbot(state: State):
    return {"messages": llm_with_tools.invoke(state["messages"])}

Define the graph:

In [ ]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[get_current_date, travily_search])
graph_builder.add_node("tools", tool_node)

## Add edges

This is a conditional edge that will only be taken if the tool condition is met:

In [ ]:
graph_builder.add_conditional_edges("chatbot", tools_condition)

If tools are used, return to the chatbot to process the tool output

In [ ]:
graph_builder.add_edge("tools", "chatbot")

Set the entry point:

In [ ]:
graph_builder.set_entry_point("chatbot")

Compile the graph

In [ ]:
graph = graph_builder.compile()

In [ ]:
from IPython.display import display, Image

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from IPython.display import Markdown

def render_in_markdown(text):
    display(Markdown(text))
    
def process_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()
    return message   

In [ ]:
def process_query(query, config=None):
    inputs = {"messages": [("user", query)]}
    message = process_stream(graph.stream(inputs, config, stream_mode="values"))
    render_in_markdown(message.content)

Is agent can remember me?

In [ ]:
process_query("My name is Ojitha")

In [ ]:
process_query("What is my name?")

As shown in the above output, model cannot remember. Therefore, need to save to the memory:

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

Now recompile and pass the memory object:

In [ ]:
graph = graph_builder.compile(checkpointer=memory)

Memory is dedicated to user session, therefore you need to pass the configuration as follows:

In [ ]:
cfg_user_1 = {"thread_id": "user_1"}
process_query("My name is Ojitha", config=cfg_user_1)

In [ ]:
process_query("What is my name?", config=cfg_user_1)